In [1]:
import numpy as np
import torch, torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import make_grid

from datetime import datetime
from torch.utils.tensorboard import SummaryWriter

import time
from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

import os

# !pip install einops
# !wget -O utils.py https://drive.google.com/uc?id=1O3QWBKrqA7s8nIGzhKMIz-YNK1-jzwml
# import utils
# from utils import plot_loss_and_accuracy

In [2]:
# Detect if we are in Google Colaboratory
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

from pathlib import Path
# Determine the locations of auxiliary libraries and datasets.
# `AUX_DATA_ROOT` is where 'notmnist.py', 'animation.py' and 'tiny-imagenet-2020.zip' are.
if IN_COLAB:
    google.colab.drive.mount("/content/drive")
    
    # Change this if you created the shortcut in a different location
    # AUX_DATA_ROOT = Path("/content/drive/My Drive/mds20_cohortney")
    
    # assert AUX_DATA_ROOT.is_dir(), "Have you forgot to 'Add a shortcut to Drive'?"
# else:
AUX_DATA_ROOT = Path(".")

Mounted at /content/drive


In [50]:
 def make_model():
    model = nn.Sequential(
        nn.Conv1d(in_channels=30, out_channels=50, kernel_size=3, padding=1), # [1, 50, 40]
        nn.BatchNorm1d(50),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size=2),   # [1, 50, 20]

        nn.Flatten(),                  # [1, 1000]
        nn.Linear(1000, 500),          # [1, 500]
        nn.Dropout(0.3),
        nn.Linear(500, 10),            # [1, 10]
        nn.LogSoftmax(dim=1)
    )
    return model
model = make_model()
model = model.to(device)

In [52]:
# model(torch.Tensor(1, 30, 40).to(device)).shape # [samples, channels, length]

In [53]:
def compute_accuracy(logits, y_true, device='cuda:0'):
    y_pred = torch.argmax(logits, dim=1)
    return (y_pred == y_true.to(device)).float().mean()

In [54]:
def accuracy(model, images, labels):
    logits = model.forward(images)
    y_pred = torch.argmax(logits, dim=1)
    return (y_pred == labels).float().mean()

In [55]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.random.permutation(len(inputs))
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [56]:
def train(model, num_epochs=2, lr=0.1, weight_decay=0, exp_name='my_network'):
    # writer = SummaryWriter(f'logs/{exp_name}')
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=7, gamma=0.1)

    train_loss = []
    val_loss = []
    train_accuracy = []
    val_accuracy = []

    for epoch in range(num_epochs):
        start_time = time.time()
        model.train(True)

        train_accuracy_batch = []

        # for batch_no, (X_batch, y_batch) in tqdm(enumerate(zip(X, y)), 
        #                                          total=len(X), desc="train " + str(epoch + 1)):
        for batch_no, (X_batch, y_batch) in enumerate(iterate_minibatches(X, y, batchsize=250, shuffle=True)):
            # main cycle
            print (X_batch.shape, y_batch.shape)
            model.zero_grad()
            X_batch_gpu = X_batch.squeeze_().to(device)
            logits = model(X_batch_gpu)
            print (logits.shape, y_batch.shape)
            loss = F.nll_loss(logits, y_batch.to(device))
            loss.backward()
            opt.step()
        
        train_accuracy_overall = np.mean(train_accuracy_batch) * 100 # mean train accuracy over 1 epoch in %
        train_accuracy.append(train_accuracy_overall.item())

    return train_accuracy_overall, 0

In [58]:
X = torch.Tensor(1000, 30, 40) # [samples, n_steps, features]
y = torch.randint(10, (1000,))

X.shape, y.shape

(torch.Size([1000, 30, 40]), torch.Size([1000]))

In [59]:
torch.unique(y)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [60]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [61]:
model = make_model()
model = model.to(device)

In [62]:
train_accuracy = -1
val_accuracy = -1

DO_TRAIN = True
if DO_TRAIN:
    %%time
    lr=0.01
    exp_name = datetime.now().isoformat(timespec='seconds') + f'exp{lr}'
    train_accuracy, val_accuracy = train(model, num_epochs=10, weight_decay=0, lr=lr, exp_name=exp_name)

    torch.save(model.state_dict(), "checkpoint.pth")
    print (train_accuracy, val_accuracy)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 7.15 µs
torch.Size([250, 30, 40]) torch.Size([250])
torch.Size([250, 10]) torch.Size([250])
torch.Size([250, 30, 40]) torch.Size([250])
torch.Size([250, 10]) torch.Size([250])
torch.Size([250, 30, 40]) torch.Size([250])
torch.Size([250, 10]) torch.Size([250])
torch.Size([250, 30, 40]) torch.Size([250])
torch.Size([250, 10]) torch.Size([250])
torch.Size([250, 30, 40]) torch.Size([250])
torch.Size([250, 10]) torch.Size([250])
torch.Size([250, 30, 40]) torch.Size([250])
torch.Size([250, 10]) torch.Size([250])
torch.Size([250, 30, 40]) torch.Size([250])
torch.Size([250, 10]) torch.Size([250])
torch.Size([250, 30, 40]) torch.Size([250])
torch.Size([250, 10]) torch.Size([250])
torch.Size([250, 30, 40]) torch.Size([250])
torch.Size([250, 10]) torch.Size([250])
torch.Size([250, 30, 40]) torch.Size([250])
torch.Size([250, 10]) torch.Size([250])
torch.Size([250, 30, 40]) torch.Size([250])
torch.Size([250, 10]) torch.Size([250])
torch.Size([

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
